# **Extracción Transformacion y Carga de datos (ETL)**

In [395]:
%config Completer.use_jedi = False
import gzip
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import ast
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\silde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\silde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [499]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('max_columns', None)
pd.set_option('max_rows', 30)
pd.options.display.max_columns

## Descripcion de Archivos:
- **steam_games.json.gz**: Contiene informacion descriptiva de cada item, como por ejemplo: precio, desarrollador, etc.
- **users_items.json.gz**: Contiene por usuario cantidad de items que compró, y tiempo de juego por item.
- **user_reviews.json.gz**: Contiene reseñas que realizaron usuarios para determinados items que compraron.

## 1. Lectura de archivos:
### Funciones para lectura y acondicionamiento de archivos.


In [396]:
# Generó una funcion para poder abrir los archivos que estan en formato comprimido gz

def leer_archivo (path):
    info = []
    for i in gzip.open(path):
        info.append(ast.literal_eval(i.decode('utf-8')))
    df = pd.DataFrame(info)
    return df


In [401]:
def desanidar_archivo(df,columna_anidada):
    
    # explode sobre columna anidada
    df_exp=df.explode(columna_anidada)
    df_exp.dropna(how='all',inplace=True)
    
    # normalizaciond de columna anidada
    normalizado = pd.json_normalize(df_exp[columna_anidada].dropna())
    normalizado.reset_index(inplace=True)
    
    #concatenacion
    df_exp.reset_index(inplace=True)
    df_salida = pd.concat([df_exp,normalizado], axis=1)
    
    #elimino la columna anidad reviews
    df_salida = df_salida.drop(columns = [columna_anidada,'index'])
    return df_salida

##  1.1  **user_reviews.json.gz** 

#### Leemos archivos y aplico funciones para desanidar y obtener formato tipo dataframe

In [477]:
df_r = leer_archivo ('user_reviews.json.gz')

La colummna reviews contiene un archivo Json anidado, aplicamos la funcion desanidar_archivo para obtener formato dataframe. 

In [478]:
final_rev = desanidar_archivo(df_r,'reviews')
final_rev.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,"Simple yet with great replayability. In my opinion does ""zombie"" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth ""zombie"" splattering fun for ..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


In [479]:
final_rev.shape

(59333, 9)

### Exploramos valores nulos

- Analisis de registros nulos:<br>
>Observamos que hay 28 nulos que corresponden a la columna que se encontraba anidada reviews.<br>
 **Transformación**: Decido eliminar los nulos porque son pocos valores.

In [480]:
final_rev.isnull().sum()

user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

In [481]:
final_rev.dropna(thresh=3,inplace=True) 
#Keep only the rows with at least 3 non-NA values.

In [482]:
final_rev.shape

(59305, 9)

### Exploramos registros duplicados

- Analisis de registros duplicados: 
>Aparecen 144 valores duplicados.<br>
> **Transformación**: Elimino duplicados.

In [483]:
data_duplicates_final_rev=  final_rev.duplicated( keep="first")
data_duplicates_final_rev.sum()

144

In [484]:
final_rev = final_rev.drop_duplicates( keep = "first")

In [485]:
final_rev.shape

(59161, 9)

### Exploración de tipo de datos de columnas.

- Analisis de tipo de datos de columnas:<br>
 **Transformación**: 
> 1. Convierto posted a formato datetime.<br> 
> 2. Agrego columna año_posted que corresponde al año de posteo

In [486]:
final_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59161 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59161 non-null  object
 1   user_url     59161 non-null  object
 2   funny        59161 non-null  object
 3   posted       59161 non-null  object
 4   last_edited  59161 non-null  object
 5   item_id      59161 non-null  object
 6   helpful      59161 non-null  object
 7   recommend    59161 non-null  object
 8   review       59161 non-null  object
dtypes: object(9)
memory usage: 4.5+ MB


In [487]:
final_rev.posted.sample(15)

48273    Posted December 14, 2013.
2054         Posted June 22, 2015.
4807         Posted April 4, 2014.
23088    Posted December 12, 2015.
486                 Posted July 9.
                   ...            
26223            Posted January 1.
46248      Posted August 24, 2011.
23560    Posted February 27, 2014.
11173    Posted February 15, 2015.
44379         Posted May 23, 2013.
Name: posted, Length: 15, dtype: object

Veo que existen algunos valores que no tienen el año de posteo. Voy a separar el año de la fecha para poder contar cuantos valores nulos de años hay.<br> Se tienen 10.091 valores de fecha sin año

In [488]:
final_rev['posted2']=final_rev['posted']

In [489]:
final_rev[['mes_dia','año']]=final_rev['posted2'].str.split(',',n=2,expand=True)
filas_con_nan = final_rev[pd.isna(final_rev['año'])]
filas_con_nan.shape

(10091, 12)

Convierto la columna posted a formato datetime y observo que obtengo nulos por los datos que no tenian año. No elimino estas filas, porque pueden tener otros campos con informacion valiosa para las funciones.

In [490]:
final_rev['posted']=final_rev['posted'].replace({'Posted':''},regex=True)
final_rev['posted']=pd.to_datetime(final_rev['posted'], errors='coerce')

In [491]:
 final_rev['posted'].isnull().sum()

10091

In [492]:
print(final_rev['posted'].max())
print(final_rev['posted'].min())

2015-12-31 00:00:00
2010-10-16 00:00:00


In [573]:
final_rev['año_posted']=final_rev['posted'].dt.year

### Feature Engineering: creo la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP

In [493]:
def analizar_sentimiento(texto):
    sia = SentimentIntensityAnalyzer()
    sentimiento = sia.polarity_scores(texto)
    
    # Determinar la etiqueta en función de la puntuación compuesta
    if sentimiento['compound'] >= 0.05:
        return 'positivo'
    elif sentimiento['compound'] <= -0.05:
        return 'negativo'
    else:
        return 'neutral'

In [494]:
final_rev['sentiment_analysis'] = final_rev['review'].apply(analizar_sentimiento)

In [495]:
final_rev[final_rev['sentiment_analysis']=='negativo'].head(1)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,posted2,mes_dia,año,sentiment_analysis
16,doctr,http://steamcommunity.com/id/doctr,,2012-11-22,,207610,No ratings yet,True,"The ending to this game is.... ♥♥♥♥♥♥♥.... Just buy it, you'll be invested, im automatically preordering season two of the walking dead game.","Posted November 22, 2012.",Posted November 22,2012.,negativo


In [496]:
final_rev.to_csv('final_rev', index = False)
final_rev.to_parquet('final_rev', index = False)

## 1.2 **users_items.json.gz**

#### Leemos archivos y aplico funciones para desanidar y obtener formato tipo dataframe

In [ ]:
df_i = leer_archivo ('users_items.json.gz')

La colummna items contiene un archivo Json anidado, aplicamos la funcion desanidar_archivo para obtener formato dataframe. 

In [497]:
final_items = desanidar_archivo(df_i,'items')

In [404]:
final_items.shape

(5170015, 8)

### Exploramos valores nulos

- Analisis de registros nulos:<br>
>Observamos que hay 16806 nulos que corresponden a la columna que se encontraba anidada items.<br>
 **Transformación**: Elimino los nulos porque corresponde a filas que tienen muchos nulos cada una y no puedo imputar.

In [420]:
final_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [418]:
final_items.dropna(thresh=5,inplace=True)

In [419]:
final_items.shape

(5153209, 8)

### Exploramos registros duplicados

- Analisis de registros duplicados: 
>Aparecen 90386 valores duplicados.<br>
> **Transformación**: Elimino duplicados.

In [450]:
data_duplicates_final_items=  final_items.duplicated( keep="first")
data_duplicates_final_items.sum()

90386

In [451]:
final_items = final_items.drop_duplicates( keep = "first")

In [452]:
final_items.shape

(5062823, 8)

In [ ]:
final_items.to_csv('final_items.csv', index = False)
final_items.to_parquet('final_items.parquet', index = False)

## **steam_games.json.gz**

In [ ]:
df_s=pd.read_json('steam_games.json.gz',compression='gzip', lines=True)

In [453]:
df_s.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}...",277.0


Al leer archivo steam, veo que tiene columnas al final correspondientes a los items que consumio cada usario. Esas columas las elimino porque las tengo en el dataframe items y no tiene sentido que esten en este dataframe. Este contiene la descripcion de cada item, y no tendria sentido tener en cada item un usuario con los items que compro

In [455]:
df_steam = df_s.drop(columns = ['items','user_id','steam_id','items_count'])

In [456]:
df_steam.shape

(120445, 15)

### Exploramos valores nulos

- Analisis de registros nulos:<br>
>Observamos que hay muchos nulos. Voy a eliminar solo las filas que tengan todas las columnas con valor nulo.<br>
 **Transformación**: Elimino los nulos.

In [462]:
df_steam.isnull().sum()

publisher        8052
genres           3283
app_name            2
title            2050
url                 0
                ...  
price            1377
early_access        0
id                  2
metascore       29458
developer        3299
Length: 15, dtype: int64

In [463]:
df_steam.dropna(inplace=True,how ='all')

In [464]:
df_steam.shape

(32135, 15)

### Exploramos registros duplicados

- Analisis de registros duplicados: 
>Aparecen 0 valores duplicados.<br>
> **Transformación**:-

In [465]:
data_duplicates_df_steam=  final_items.duplicated( keep="first")
data_duplicates_df_steam.sum()

0

### Exploración de tipo de datos de columnas.

- Analisis de tipo de datos de columnas:<br>
 **Transformación**: 
> 1. Convierto precio a formato float.<br>
> 2. Convierto release_date a formato datetime
> 3. Agrero columna año de release_date

#### Precio

In [466]:
df_steam.price.unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

In [467]:
precio_cero=['Free To Play', 'Free to Play','Free','Free Mod','Free HITMAN™ Holiday Pack','Play the Demo','Free to Use','Free Demo','Play WARMACHINE: Tactics Demo','Play for Free!','Free to Try','Free Movie']
precio_error=['Install Now','Install Theme','Third-party','Play Now'] 
precio_cor = ['Starting at $499.00','Starting at $449.00']

In [468]:
df_steam['price'] = df_steam['price'].replace(precio_cero, 0)
df_steam['price'] = df_steam['price'].replace(precio_error, np.nan)
df_steam['price'] = df_steam['price'].replace(precio_cor, 499.00)

In [469]:
df_steam.price.unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00, 3.9900e+00,
       9.9900e+00, 1.8990e+01, 2.9990e+01,        nan, 1.0990e+01,
       1.5900e+00, 1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00,
       6.9900e+00, 7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00,
       1.2990e+01, 5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00,
       2.4990e+01, 1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01,
       1.3990e+01, 3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01,
       9.9990e+01, 1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01,
       4.9990e+01, 5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01,
       1.1999e+02, 1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01,
       8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01,
       5.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02,
       1.2900e+00, 3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01,
       4.9000e+01, 2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01,
       4.4900e+00, 2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e

In [470]:
df_steam['price']=df_steam['price'].astype(float)

In [471]:
df_steam['price'].isnull().sum()

1383

### release_date

In [472]:
df_steam['release_date']=pd.to_datetime(df_steam['release_date'],errors='coerce')

In [473]:
df_steam['release_date'].isnull().sum()

2241

In [474]:
df_steam['release_date'].max()

Timestamp('2021-12-31 00:00:00')

In [475]:
df_steam['release_date'].min()

Timestamp('1970-07-15 00:00:00')

In [476]:
df_steam['year']=df_steam['release_date'].dt.year

df_steam.to_csv('df_steam.csv', index = False)
df_steam.to_parquet('df_steam.parquet', index = False)

# **Desarrollo de datasets para funciones**

#### Voy a realizar uniones en los dataset para luego poder realizar las transformaciones que me permiten obtener los dataframe para alimentar las funciones

### 1) df_items_steam : es la union de final_items con df_steam

Voy a unir el dataset final_items (cantidad de items por usuario) con el de la descripcion de los items (df_steam), por el campo item_id.  
Es decir me voy a traer al dataset final_items la descripcion de cada item que esta en df_steam

In [ ]:
final_items ['item_id'] = final_items ['item_id'].astype(float) 

In [ ]:
df_items_steam = final_items.merge(df_steam, left_on='item_id',right_on='id',how='left')

df_items_steam.drop(columns=['index'], inplace=True)

In [580]:
df_items_steam.shape

In [502]:
df_items_steam.isnull().sum()

level_0                   0
user_id                   0
items_count               0
steam_id                  0
user_url                  0
level_0               33612
item_id               33612
item_name             33612
playtime_forever      33612
playtime_2weeks       33612
publisher           1012477
genres               965202
app_name             875758
title                947543
url                  858952
release_date         954196
tags                 875875
reviews_url          892564
discount_price      5165453
specs                891572
price                944228
early_access         858952
id                   892564
metascore           2464940
developer            981917
year                 954196
dtype: int64

### 2) df_items_rev: es la union de final_items con final_rev

Voy a unir el dataset final_items  (dataset con items por usuario) con el dataset final_rev que tiene las reseñas.
La union debe hacerse usando don claves: user_id y item_id

In [622]:
final_rev['user_id'] = final_rev['user_id'].str.strip()
final_items['user_id']= final_items['user_id'].str.strip()
final_rev['item_id'] = final_rev['item_id'].astype(float)
final_items ['item_id'] = final_items ['item_id'].astype(float) 

In [623]:
df_items_rev=pd.merge(final_items,final_rev, left_on=['user_id','item_id'], right_on=['user_id','item_id'], how='left')

In [624]:
df_items_rev.shape

(5170687, 20)

In [626]:
df_items_rev.isnull().sum()

user_id                     0
items_count                 0
steam_id                    0
user_url_x                  0
item_id                 16806
item_name               16806
playtime_forever        16806
playtime_2weeks         16806
user_url_y            5153207
funny                 5153207
posted                5155900
last_edited           5153207
helpful               5153207
recommend             5153207
review                5153207
posted2               5153207
mes_dia               5153207
año                   5155900
sentiment_analysis    5153207
año_posted            5155900
dtype: int64

In [627]:
df_items_rev.dropna(subset=['recommend'],inplace=True)

In [628]:
df_items_rev.shape

(17480, 20)

### 3) df_items_steam_rv : es la union de df_items_steam con final_rev

Voy a unir el dataset df_items_steam (dataset con items por usuario e informacion de cada item) generado en el paso anterior con el dataset final_rev
que tiene las reseñas. La union debe hacerse usando don claves: user_id y item_id

In [504]:
# Acomodo el formato de las columnas para poder hacer el merge
final_rev['user_id'] = final_rev['user_id'].str.strip()
df_items_steam ['user_id'] = df_items_steam ['user_id'].str.strip()
final_rev['item_id'] = final_rev['item_id'].astype(float)


In [240]:
df_items_steam_rv=pd.merge(df_items_steam,final_rev, left_on= ['user_id','item_id'], right_on=['user_id','item_id'], how='left')

In [241]:
df_items_steam_rv.shape

(5187569, 39)

In [242]:
df_items_steam_rv.isnull().sum()

level_0              0
user_id              0
items_count          0
steam_id             0
user_url_x           0
                ...   
review         5170013
posted2        5170013
mes_dia        5170013
año            5172724
sentimiento    5170013
Length: 39, dtype: int64

In [ ]:
# Elimino los nulos y me quedo solo con los items que tienen review en este dataset

In [629]:
df_items_steam_rv.dropna(subset=['posted'],inplace=True)

In [630]:
df_items_steam_rv.shape

(14845, 39)

## FUNCION 1:
### def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

#### Voy a generar un dataset con los gastos por usuario que luego será llamado por la función. 

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam

In [506]:
df_userdata1 = df_items_steam[['user_id','item_id','price']]

Realizo agrupacion por usuario y aplico la funcion de agregación suma a precio para calcular el gasto

In [507]:
df_userdata1_agg=df_userdata1.groupby('user_id').agg(  gasto = ('price','sum')).reset_index()

In [508]:
df_userdata1_agg.head(3)

,user_id,gasto
0,--000--,874.52
1,--ace--,310.87
2,--ionex--,309.82


Guardo el dataset a un csv para luego alimentar a la función.

In [509]:
df_userdata1_agg.to_csv('userdata_1')

#### Voy a generar un dataset que tenga por usuario el porcentaje de recomendación.

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam_rv

In [632]:
df_userdata2 = df_items_rev[['user_id','recommend','items_count']]

Realizo agrupacion por usuario y aplico la funcion de agregación suma a recommend para obtener el total de recomendaciones por usuario, la funcion maximo para items_count, que igualmente es constante para todos los valores de mismo usuario. Con esos dos valores puedo calcular el porcentaje de recomendacion. 

In [633]:
df_userdata2_agg =df_userdata2 .groupby('user_id').agg(recomendado = ('recommend','sum'),totalit=('items_count','max') ).reset_index()

In [634]:
df_userdata2_agg['recomendado'] = df_userdata2_agg['recomendado'].astype(int)

In [635]:
df_userdata2_agg['porcentaje']=(df_userdata2_agg['recomendado']/df_userdata2_agg['totalit'])*100

In [636]:
df_userdata2_agg.head(4)

,user_id,recomendado,totalit,porcentaje
0,--000--,1,58,1.724138
1,-2SV-vuLB-Kg,1,68,1.470588
2,-Azsael-,1,167,0.598802
3,-Kenny,1,118,0.847458


Guardo el dataset a un csv para luego alimentar a la función.

In [637]:
df_userdata2_agg.to_csv('userdata_2')

### Genero la función 1:

In [659]:
def userdata (df1,df2,usuario):
    user = usuario
    df1_filtrado = df1[df1['user_id'] == user]
    df2_filtrado = df2[df2['user_id'] == user]
    valor1=df1_filtrado.gasto.iloc[0]
    valor2=df2_filtrado.porcentaje.iloc[0]
    return  ({'dinero gastado' : valor1,'porcecntaje_recom':valor2})

In [660]:

userdata(df_userdata1_agg,df_userdata2_agg,'76561197970982479')

{'dinero gastado': 3424.3099999999854, 'porcecntaje_recom': 1.083032490974729}

## FUNCION 2
## def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

#### Voy a generar un dataset que tenga por fecha de posteo, el usuario, recommend y la cantidad de items por usuario.

Genero un dataset para realizar los calculos tomando las columnas que necesito del final_rev.

In [642]:
df_countreviews= final_rev[['posted','user_id','recommend']].copy()

In [643]:
df_countreviews['recommend']=df_countreviews['recommend'].astype(int)

In [644]:
df_countreviews.dropna(subset=['user_id'],inplace=True)

Guardo el dataset a un csv para luego alimentar a la función.

In [646]:
df_countreviews.to_csv('counterviwes')

### genero la Función 2:

In [661]:
def countreviews(df,fecha_inicio, fecha_fin):
    # Convierte las fechas a objetos DateTime si no lo están
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    # Filtra el DataFrame para incluir solo filas dentro del rango de fechas
    df_filtrado = df[(df['posted'] >= fecha_inicio) & (df['posted'] <= fecha_fin)]
    
    # Calcula cantidad de usuarios y porcentaje de recomendacion
    cantidad_usuarios = df_filtrado['user_id'].nunique()
    recomendacion = df_filtrado['recommend'].sum()/df_filtrado['recommend'].count()
    # Puedes agregar más columnas según tus necesidades
    
    # Retorna los valores promedio calculados
    return {'cantidad_usuarios': cantidad_usuarios,'recomendacion': recomendacion}
   

In [663]:
fecha_inicio = '2011-07-15'
fecha_fin = '2012-07-15'
resultado = countreviews(df_countreviews, fecha_inicio, fecha_fin)
resultado
# Imprime más resultados si es necesario

{'cantidad_usuarios': 617, 'recomendacion': 0.981081081081081}

## FUNCION 3
## def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

### Voy a generar un dataset que tenga en una columna todos los generos y en otra columna la cantidad de horas de cada uno (playtimeForever)

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam

In [523]:
df_genre = df_items_steam [['genres','playtime_forever']]

Como cada item contiene una lista de generos, aplano la columna genero utilizando la función explode.

In [524]:
df_genre_exp=df_genre.explode('genres')

In [525]:
df_genre_exp.shape

(10992761, 2)

In [526]:
df_genre_exp.isnull().sum()

genres              965202
playtime_forever     50418
dtype: int64

Elimino los valores nulos de genero

In [133]:
df_genre_exp.dropna(inplace=True)

Exploro la cantidad de valores unicos de genero y cuantos registros tengo de cada uno.

In [500]:
df_genre_exp.genres.value_counts()

Action                       2808730
Indie                        1764035
Adventure                    1166043
RPG                           911629
Strategy                      803709
Free to Play                  754561
Simulation                    536340
Casual                        500418
Massively Multiplayer         372342
Early Access                  123862
Sports                        108578
Racing                        102615
Utilities                       9816
Design &amp; Illustration       8692
Animation &amp; Modeling        7160
Video Production                4607
Web Publishing                  3857
Education                       3096
Software Training               2381
Audio Production                 880
Photo Editing                    596
Name: genres, dtype: int64

Agrupo por genero aplicando la funcion de agregación suma sobre la columna playTime_forever, obteniendo la cantidad de horas totales por genero.

In [527]:
df_genre_exp_agg =df_genre_exp.groupby('genres').agg(  horas = ('playtime_forever','sum'))

Ordeno el dataset de mayor a menor para obtener del indice la posicion del ranking y reseteo indice

In [528]:
df_genre_exp_agg.sort_values(by=['horas'],ascending = False,inplace=True)

In [530]:
df_genre_exp_agg.reset_index(inplace=True)

In [531]:
df_genre_exp_agg

,genres,horas
0,Action,3.113563e+09
1,Indie,1.494622e+09
2,RPG,1.041023e+09
3,Adventure,9.099951e+08
4,Simulation,8.676463e+08
5,Strategy,6.593638e+08
6,Free to Play,6.107529e+08
7,Massively Multiplayer,4.465941e+08
8,Casual,2.522329e+08
9,Early Access,1.587013e+08


Guardo el dataset a un csv para luego alimentar a la función.

In [533]:
df_genre_exp_agg.to_csv('genre')

### Genero la Funcion 3:

In [534]:
def genre(df,genero):
    gen=genero
    posicion = df_genre_exp_agg[df_genre_exp_agg['genres'] == gen].index[0]
    return {
        'posicion en ranking': posicion + 1
    }

In [664]:
resultado = genre(df_genre_exp_agg,'Software Training')
resultado


{'posicion en ranking': 18}

## FUNCION 4
## def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

### Voy a generar un dataframe que tenga user_id, user_url, genre y playtime_forever

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_items_steam

In [535]:
df_userforgenre = df_items_steam[['user_id','user_url','genres','playtime_forever']]

In [536]:
df_userforgenre_exp=df_userforgenre.explode('genres')

In [537]:
df_userforgenre_exp.dropna(subset=['genres'],inplace=True)

Agrupo por usuario y  por genero y aplico sobre playtime_forever la funcion suma para calcular las horas totales y para user_url dejo el primer valor ya que es constante por usuario. 

In [538]:
df_agg= df_userforgenre_exp. groupby (['user_id', 'genres']).agg (horas=('playtime_forever','sum'),url=('user_url','first')). reset_index ()

In [539]:
df_agg.head(15)

,user_id,genres,horas,url
0,--000--,Action,51389.0,http://steamcommunity.com/id/--000--
1,--000--,Adventure,5270.0,http://steamcommunity.com/id/--000--
2,--000--,Casual,3026.0,http://steamcommunity.com/id/--000--
3,--000--,Early Access,2533.0,http://steamcommunity.com/id/--000--
4,--000--,Free to Play,3816.0,http://steamcommunity.com/id/--000--
5,--000--,Indie,37254.0,http://steamcommunity.com/id/--000--
6,--000--,Massively Multiplayer,3327.0,http://steamcommunity.com/id/--000--
7,--000--,RPG,13297.0,http://steamcommunity.com/id/--000--
8,--000--,Racing,26768.0,http://steamcommunity.com/id/--000--
9,--000--,Simulation,1855.0,http://steamcommunity.com/id/--000--


Guardo el dataset a un csv para luego alimentar a la función.

In [540]:
df_agg.to_csv('userforgenre')

### Genero la Funcion 4:

In [665]:
def userforgenre (df,genero):
   
    gen=genero
    df_filtrado = df[df['genres'] == gen]  
    df_filtrado.sort_values(by=['horas'],ascending=False,inplace=True)
    df_top5 = df_filtrado[['user_id','url']].head(5)
    js= df_top5.to_json(orient = 'records') 
    return js


In [666]:
userforgenre(df_agg,'Action')

<ipython-input-665-550439e61db6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.sort_values(by=['horas'],ascending=False,inplace=True)


'[{"user_id":"DeEggMeister","url":"http:\\/\\/steamcommunity.com\\/id\\/DeEggMeister"},{"user_id":"inven","url":"http:\\/\\/steamcommunity.com\\/id\\/inven"},{"user_id":"Outbackalien","url":"http:\\/\\/steamcommunity.com\\/id\\/Outbackalien"},{"user_id":"kitty_47","url":"http:\\/\\/steamcommunity.com\\/id\\/kitty_47"},{"user_id":"Brizbrenz","url":"http:\\/\\/steamcommunity.com\\/id\\/Brizbrenz"}]'

## FUNCION 5:
## def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:

### Voy a generar un dataset para la funcion que contenga, empresa desarrolladora, porcentaje de contendido libre

Genero un dataset para realizar los calculos tomando las columnas que necesito del df_steams

In [560]:
df_developer=df_steam[['id','developer','price','year']]

Genero una columna que ponga un 1 si el precio es 0 para identificar el contenido Free

In [561]:
#df_developer['is_free']=df_developer.apply(lambda x: 1 if x['price']==0 else 0,axis=1)
df_developer.loc[:, 'is_free'] = df_developer.apply(lambda x: 1 if x['price'] == 0 else 0, axis=1)

C:\Users\silde\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [547]:
df_developer

,id,developer,price,year,is_free
88310,761140.0,Kotoshiro,4.99,2018.0,0
88311,643980.0,Secret Level SRL,0.00,2018.0,1
88312,670290.0,Poolians.com,0.00,2017.0,1
88313,767400.0,彼岸领域,0.99,2017.0,0
88314,773570.0,None,2.99,NaN,0
...,...,...,...,...,...
120440,773640.0,"Nikita ""Ghost_RUS""",1.99,2018.0,0
120441,733530.0,Sacada,4.99,2018.0,0
120442,610660.0,Laush Dmitriy Sergeevich,1.99,2018.0,0
120443,658870.0,"xropi,stev3ns",4.99,2017.0,0


Agrupo por desarrollador, por año y aplico suma a columna free_it, y la user_url me quedo con la primera ya que son todas iguales.

In [548]:
#df_agg= df_userforgenre_exp. groupby (['user_id', 'genres']).agg (horas=('playtime_forever','sum'),url=('user_url','first')). reset_index ()
df_dv_agg=df_developer. groupby (['developer','year']).agg(todos_it = ('is_free','count'),free_it = ('is_free','sum')). reset_index ()

Al dataframe agrupado le agrego una columna con el calculo del porcenjaje.

In [549]:
df_dv_agg['porcentaje_free'] = (df_dv_agg ['free_it']/df_dv_agg ['todos_it'])*100

Genero un dataset de salida para la funcion que tenga las columnas necesarias, desarrollador, año y contenido libre

In [551]:
df_salida =df_dv_agg[['developer','year','porcentaje_free']].copy()

In [562]:
df_salida

,developer,year,porcentaje_free
12300,Team B.R.O.S.,2017.0,100.0
9544,PixelSoft,2017.0,100.0
12706,Tigerish Games,2015.0,100.0
2334,Chad Meffert,2017.0,100.0
5827,Holy Priest,2015.0,100.0
...,...,...,...
5206,Gato Salvaje S.L.,2015.0,0.0
5207,Gato Salvaje S.L.,2016.0,0.0
5208,Gaute Heggen,2017.0,0.0
5209,Gaweb Studio,2016.0,0.0


Guardo el dataset a un csv para luego alimentar a la función.

In [566]:
df_salida.to_csv('developer.csv')

### Genero la Funcion 5:

In [671]:
def developer (df,desarrollador):
   
    des=desarrollador
    df_filtrado = df[df['developer'] == des]  
    df_dev=df_filtrado[['year','porcentaje_free']]
    js= df_dev.to_json(orient = 'records')
    return js

In [672]:
developer(df_salida,'Activision')

'[{"year":1993.0,"porcentaje_free":0.0},{"year":1996.0,"porcentaje_free":0.0},{"year":1997.0,"porcentaje_free":0.0},{"year":2000.0,"porcentaje_free":0.0}]'

## FUNCION 6
## def sentiment_analysis( año : int ): Según el año de lanzamiento(posted), se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.


### Voy a generar un dataset para la funcion que contenga, año de posteo y una columa por cada categoria de sentiment_analysis

In [575]:
final_rev.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'review', 'posted2', 'mes_dia', 'año',
       'sentiment_analysis', 'año_posted'],
      dtype='object')

In [577]:
df_sentiment_analysis=final_rev[['posted','año_posted','sentiment_analysis']]

voy a generar dummies para la columna sentimient_analysis

In [578]:
df_sentiment_analysis_dummies = pd.get_dummies( df_sentiment_analysis,columns=['sentiment_analysis'], 
                            drop_first = False, dtype=int)

In [580]:
df_sentiment_analysis_dummies.head(2)

,posted,año_posted,sentiment_analysis_negativo,sentiment_analysis_neutral,sentiment_analysis_positivo
0,2011-11-05,2011.0,0,0,1
1,2011-07-15,2011.0,0,0,1


Agrupo por año para obtener el dataset final para alimentar la funcion. 

In [585]:
df_sentiment_analysis_dummies_agg=df_sentiment_analysis_dummies. groupby (['año_posted']).agg(positivo = ('sentiment_analysis_positivo','sum'),negativo = ('sentiment_analysis_negativo','sum'), neutral=('sentiment_analysis_neutral','sum')). reset_index ()

In [587]:
df_sentiment_analysis_dummies_agg.head(10)

,año_posted,positivo,negativo,neutral
0,2010.0,48,10,8
1,2011.0,397,61,72
2,2012.0,832,144,233
3,2013.0,4671,756,1350
4,2014.0,14218,3308,4534
5,2015.0,11153,3372,3903


Guardo el dataset a un csv para luego alimentar a la función.

In [588]:
df_sentiment_analysis_dummies_agg.to_csv('sentiment_analysis')

### Genero la Funcion 6:

In [674]:
def sentiment_analysis(df,año):
    year=año
    df_s = df[df['año_posted'] == year]
    js= df_s.to_json(orient = 'records')
    return js

In [675]:
sentiment_analysis(df_sentiment_analysis_dummies_agg,2014)

'[{"a\\u00f1o_posted":2014.0,"positivo":14218,"negativo":3308,"neutral":4534}]'